In [1]:
# This needs to be rewritten. Currently clips are taken and analyzed in 2 second chunks. 
# Audio needs to come in as a stream. Every X number of milliseconds OR after the FFT/Prediction is completed, the next
# 2048 samples need to be taken from the stream and analyzed. 

# https://forum.pjrc.com/threads/24492-Using-the-PDB-on-Teensy-3#post_message_37659
# https://forum.pjrc.com/threads/24611-Problem-with-DMA-interrupt
# https://forum.pjrc.com/threads/18237-teensy-3-0-memory-to-memory-DMA-help

# https://python-sounddevice.readthedocs.io/en/0.3.11/examples.html

In [1]:
import tinn
import random
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import Queue
from threading import Timer
from threading import Thread
from threading import Event

In [3]:
#https://pymotw.com/2/threading/

In [2]:
class Data:
    def __init__(self, path, nips, nops):
        self.read_data(path, nips, nops)

    def __repr__(self):
        return str( str(self) +'rows with'+ str(len(self.in_[0])) +'inputs and'+ str(len(self.tg[0])) +'outputs' )
        #return f'{len(self)} rows with {len(self.in_[0])} inputs and {len(self.tg[0])} outputs.'

    def read_data(self, path, nips, nops):
        self.in_, self.tg = [], []
        with open(path) as data_file:
            for line in data_file:
                row = list(map(float, line.split()))
                self.in_.append(row[:nips])
                self.tg.append(row[nips:])

    def shuffle(self):
        indexes = list(range(len(self.in_)))
        random.shuffle(indexes)
        self.in_ = [self.in_[i] for i in indexes]
        self.tg = [self.tg[i] for i in indexes]

    def __len__(self):
        return len(self.in_)
    
    def dLen(self):
        return len(self.in_)

In [3]:
class DataOnTheFly:
    def __init__(self, arr, nips, nops):
        self.read_array(arr, nips, nops)
        
    def read_array(self, arr, nips, nops):
        self.in_, self.tg = [], []

        self.in_.append(arr[:nips])
        self.tg.append(arr[nips:])
        

In [4]:
def train(_nips, _nhid, _nops, _rate, _anneal, _data, _iter):
    nips = _nips
    nhid = _nhid
    nops = _nops
    rate = _rate
    anneal = _anneal
    
    data = Data(_data, nips, nops)
    
    t = tinn.Tinn(nips, nhid, nops)
    
    for i in range(_iter):
        data.shuffle()
        error = 0
        for in_, tg in zip(data.in_, data.tg):
            error += tinn.xttrain(t, in_, tg, rate)
        print 'Error: ' + str(error/len(data)) + ' learning rate: ' + str(rate) + ' Iteration: ' + str(i)
        rate *= anneal

    t.save('savedApp.tinn')

In [5]:
def predict(arr, _nips, _nops):

    data = DataOnTheFly(arr, _nips, _nops)
    
    loaded = tinn.xtload('savedApp.tinn')

    in_ = data.in_[0]
    tg = data.tg[0]
        
    pd = tinn.xtpredict(loaded, in_)
        
    #print(' '.join(map(str, tg)))
    #print(' '.join(map(str, pd)))
    
    return pd

In [8]:
#https://stackoverflow.com/questions/12435211/python-threading-timer-repeat-function-every-n-seconds
#Repeated Timer
class MyThread(Thread):
    def __init__(self, event):
        Thread.__init__(self)
        self.stopped = event

    def run(self):
        #while not self.stopped.wait(0.1):
        while True:
            #Call a function
            fastThread()

In [9]:
def getAudio():
    fs = 1024
    duration = 2
    
    myRecording1 = sd.rec(duration * fs, samplerate=fs, channels=1)
    sd.wait()
    
    #Each sample is of type list. Convert from list to int
    bClip = []
    for i in range(len(myRecording1)):
        bClip.append( myRecording1[i][0] )
    
    return bClip

In [10]:
def fastThread():
    #Get clip
    aClip = getAudio()
    
    #Push to queue
    processQueue.put( aClip )
    
    #Maybe delay a split second - Done in Thread Class

In [6]:
def getFFT(_aClip):
    
    fs = 1024
    duration = 2
    
    #Get data
    #data = aClips[i]
    data = _aClip

    #Smooth the FFT by windowing data
    data = data * np.hanning(len(data))

    #Preform FFT and take only real data
    fft = abs(np.fft.fft(data).real)

    #Keep only first half
    fft = fft[:int(len(fft))/2]

    #Create Frequency list
    freq = np.fft.fftfreq( (fs * duration) ,1.0/fs)

    #Keep only first half
    freq = freq[:int(len(freq))/2] 

    #At this point, the length of the fft/freq is cut in reduced from 2048 to 1024. But the cut off frequency is actually 512.
    #The freq axis shows why. Values are now 0, 0.5, 1, 1.5Hz...
    #Therefore drop every other value
    fft = fft[0::2]
    freq = freq[0::2]

    #Save for later
    #ffts.append( fft )
    
    return fft

In [7]:
def formatDataForPrediction(arr):
    #Convert to list
    arr = arr.tolist()
    
    #Append 1 0 to list - woof
    arr = arr + [1.0, 0.0]
    
    return arr

In [8]:
#Train NN
train(512, 32, 2, 0.5, 0.99, 'woofsAndMeows.data', 64)

Error: 0.275107626353 learning rate: 0.5 Iteration: 0
Error: 0.191554700351 learning rate: 0.495 Iteration: 1
Error: 0.135612863625 learning rate: 0.49005 Iteration: 2
Error: 0.0890952167087 learning rate: 0.4851495 Iteration: 3
Error: 0.0541806731626 learning rate: 0.480298005 Iteration: 4
Error: 0.0380856989326 learning rate: 0.47549502495 Iteration: 5
Error: 0.0268200078773 learning rate: 0.4707400747 Iteration: 6
Error: 0.0197721742512 learning rate: 0.466032673953 Iteration: 7
Error: 0.016292473158 learning rate: 0.461372347214 Iteration: 8
Error: 0.0131018430986 learning rate: 0.456758623742 Iteration: 9
Error: 0.0109796204628 learning rate: 0.452191037504 Iteration: 10
Error: 0.00944751636389 learning rate: 0.447669127129 Iteration: 11
Error: 0.00815375638722 learning rate: 0.443192435858 Iteration: 12
Error: 0.00725580452073 learning rate: 0.438760511499 Iteration: 13
Error: 0.00650047624576 learning rate: 0.434372906384 Iteration: 14
Error: 0.00589379955764 learning rate: 0.43

In [14]:
'''
#Create queues
processQueue = Queue.Queue()

#Create events
stopFlag = Event()

#Create threads
thread = MyThread(stopFlag)
thread.start()

# this will stop the timer
#stopFlag.set()
'''

'\n#Create queues\nprocessQueue = Queue.Queue()\n\n#Create events\nstopFlag = Event()\n\n#Create threads\nthread = MyThread(stopFlag)\nthread.start()\n\n# this will stop the timer\n#stopFlag.set()\n'

In [12]:
#Setup
fs = 1024
stream = sd.InputStream(samplerate=fs, channels=1)
stream.start()

In [ ]:
#While loop
while True:
    
    #Wait for something in Queue
    #if not processQueue.empty(): 
    if True:
        
        #aClip = processQueue.get()

        #Get FFT
        #clipFFT = getFFT( aClip )

        aClip = stream.read(2048)[0].tolist()
        
        #Each sample is of type list. Convert from list to int
        bClip = []
        for i in range(len(aClip)):
            bClip.append( aClip[i][0] )
        
        
        #print type(bClip), len(bClip), bClip
        
        clipFFT = getFFT( bClip )

        #Format for NN
        formattedClip = formatDataForPrediction(clipFFT)

        ##Predict
        predictedValues = predict( formattedClip, 512, 2 )
        #print predictedValues[0], predictedValues[1]

        #Found woof
        if predictedValues[0] > 0.8 and predictedValues[1] < 0.1:
            print 'Found Woof'
        #Found meow   
        elif predictedValues[0] > 0.4 and predictedValues[1] > 0.7:
            print 'Meow'

print 'Done'

Found Woof
Found Woof


In [11]:
stream.stop()